In [1]:
import os, boto3, requests, io, json
from io import StringIO
from datetime import datetime, timedelta, timezone
import pandas as pd

In [2]:
with open("credentials/aws_keys.json") as f:
    keys = json.load(f)

os.environ["AWS_ACCESS_KEY_ID"] = keys["AWS_ACCESS_KEY_ID"]
os.environ["AWS_SECRET_ACCESS_KEY"] = keys["AWS_SECRET_ACCESS_KEY"]
os.environ["AWS_DEFAULT_REGION"] = keys["AWS_DEFAULT_REGION"]

with open("credentials/ree.json") as f:
    ree = json.load(f)

os.environ["REE_API_KEY"] = ree["REE_API_KEY"]

In [3]:
API_TOKEN = os.environ["REE_API_KEY"]
#S3_BUCKET = "hab-ree-data"
S3_PREFIX = "demanda-real"
S3_BUCKET = "hab-ree-data-json"

In [4]:
s3 = boto3.client("s3")

buckets = [b["Name"] for b in s3.list_buckets()["Buckets"]]
print("Buckets:", buckets)

Buckets: ['hab-ree-data', 'hab-ree-data-json']


In [5]:
START_DATE = datetime(2014, 1, 1, tzinfo=timezone.utc)
END_DATE   = datetime.now(timezone.utc)

MESES_ES = {
    1:"enero", 2:"febrero", 3:"marzo", 4:"abril", 5:"mayo", 6:"junio",
    7:"julio", 8:"agosto", 9:"septiembre", 10:"octubre", 11:"noviembre", 12:"diciembre"
}

In [6]:
# Comprobar el indicator_id con el código de ree_indicators. Lo filtré para ver los que contienen "demanda real"

url = "https://api.esios.ree.es/indicators/1293"
headers = {
    "x-api-key": API_TOKEN,
    "Accept": "application/json",
    "Content-Type": "application/json"
}

In [7]:
def s3_object_exists(bucket, key):
    try:
        s3.head_object(Bucket=bucket, Key=key)
        return True
    except s3.exceptions.NoSuchKey:
        return False
    except Exception:
        return False

In [11]:
cur = START_DATE
subidos, omitidos = 0, 0

while cur.date() <= END_DATE.date():
    yyyy = f"{cur.year:04d}"
    mm   = f"{cur.month:02d}"
    dd   = f"{cur.day:02d}"
    mes_es = MESES_ES[cur.month]

    params = {
        "start_date": f"{yyyy}-{mm}-{dd}T00",
        "end_date":   f"{yyyy}-{mm}-{dd}T23"
    }

    try:
        resp = requests.get(url, headers=headers, params=params, timeout=90)
        resp.raise_for_status()
        data = resp.json()
        values = data.get("indicator", {}).get("values", []) or []

        if values:
            #df = pd.DataFrame(values)

            #csv_buf = StringIO()
            #df.to_csv(csv_buf, index=False)
            #key = f"{S3_PREFIX}/{yyyy}/{mm}-{mes_es}/ree-data_{yyyy}{mm}{dd}.csv"

            body = json.dumps(values, ensure_ascii=False, indent=2)
            key = f"{S3_PREFIX}/{yyyy}/{mm}-{mes_es}/ree-data_{yyyy}{mm}{dd}.json"

            if s3_object_exists(S3_BUCKET, key):
                #print(f"Archivo ya existe: s3://{S3_BUCKET}/{key}")
                omitidos += 1
            else:
                s3.put_object(
                    Bucket=S3_BUCKET,
                    Key=key,
                    #Body=csv_buffer.getvalue(),
                    Body=body,
                    ContentType="application/json"
                )
                #print(f"Archivo subido: s3://{S3_BUCKET}/{key} ({len(df)} filas)")
                subidos += 1
        else:
            #print(f"Sin datos: {yyyy}-{mm}-{dd}")
            omitidos += 1

    except requests.HTTPError as e:
        code = getattr(e.response, "status_code", "NA")
        print(f"HTTP {code} en {yyyy}-{mm}-{dd}: {e}")
        omitidos += 1
    except Exception as e:
        print(f"Error general en {yyyy}-{mm}-{dd}: {e}")
        omitidos += 1

    # Siguiente día
    cur = cur + timedelta(days=1)
    # time.sleep(0.1)

print(f"Resumen: \nArchivos subidos: {subidos} \nArchivos omitidos, ya existentes o sin datos: {omitidos}")

Resumen: 
Archivos subidos: 4271 
Archivos omitidos, ya existentes o sin datos: 0
